<a href="https://colab.research.google.com/github/sunnygong369/Covisitation-Matrix/blob/main/MOVIE_MATRIX_DONE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Settings**

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_ratings.csv') #load the csv file --> the "history table"
df = df[["userId", "movieId", "timestamp"]]
df = df.reset_index(drop = True)
df.head(10)

### **Rearranging the dataset**

*   **Sorting by timestamp [to make order matter]**

In [ ]:
df_grp = df.groupby('userId') #group by "userId"

df_grp_srt = df_grp.apply(lambda x: x.sort_values('timestamp')) #sorting each group by timestamp (ordered)

df_grp_srt

In [ ]:
#quick check on timestamp sorting:
df_grp_srt[df_grp_srt['userId'] == 99] #replace 99 with any number

*   **Filtering out the timestamp column**

In [34]:
df_new = zip(df_grp_srt['userId'].tolist(),df_grp_srt['movieId'].tolist()) #put 'userId' and 'movieId' to list
df_new = pd.DataFrame(df_new)
df_new.columns = ['User ID', 'Movie ID'] #rename the columns
df_new.index.name = 'Index'

df_new

,User ID,Movie ID
Index,,
0,1,2762
1,1,54503
2,1,112552
3,1,96821
4,1,5577
...,...,...
493920,5108,2648
493921,5108,318
493922,5108,1219


------------------------------------------------------------------------------
*   **Step abandoned [crashed for trying to display entries in "user1 item1 item2 item3 ... itemN" format]**
------------------------------------------------------------------------------


In [ ]:
# aimed to arrange data in the format as such: "user1 item1 item2 item3 ... itemN" for all the rows
#it will crash --> don't run
#it will crash --> don't run
#it will crash --> don't run

df_new_wide = df_new.pivot_table(index='User ID', columns='Index', values='Movie ID', aggfunc='first') 

df_new_wide

*   **Rearranging the entries**

In [ ]:
df_new1 = df_new.groupby('User ID')['Movie ID'].agg(list).reset_index() #take the "User ID" and "Movie ID" columns to create the users' history table
df_new1

In [ ]:
df_new2 = df_new1["Movie ID"] #select "Movie ID" --> to creat pairs at the next stage
df_new2

*   **Finding all the pairs from the entries** (time: 3s + 3s from the last run)

In [ ]:
from nltk import bigrams
bigrams_list = [list(bigrams(i)) for i in df_new2] #finding all the pairs
bigrams_list

bigrams_list = [[str(i) for i in sublist] for sublist in bigrams_list] #making all the pairs to string (for counting purposes)
bigrams_list

In [ ]:
bigrams_list = [[i.replace('(', '').replace(')', '') for i in sublist] for sublist in bigrams_list] #remove all brackets
bigrams_list

*   **Counting all the pairs**

In [ ]:
from collections import Counter

all_pairs = [i for sublist in bigrams_list for i in sublist] #list of lists unpacked

pairs_counter = Counter(all_pairs) #counting unique pairs from "history" (i.e. ordered entries)
pairs_counter

### **Constructing a [blank] Covisitation Matrix**

*   **Unique IDs for labels**

In [ ]:
unique_val = pd.DataFrame(pd.unique(df_new2.explode())) #unpack and collect unique values -> then make it a df
unique_val_srt = unique_val[0].sort_values().astype(str).tolist() #sorting df by unique values -> making them strings (for labeling purposes) in list
unique_val_srt

*   **Blank matrix filled with zeros**

In [ ]:
co_matrix = np.zeros((len(unique_val_srt), len(unique_val_srt)), dtype=int) #a blank covisitation matrix created
co_matrix = pd.DataFrame(co_matrix) #converting matrix to df
co_matrix.index = unique_val_srt #labeling the rows with unique IDs
co_matrix.columns = unique_val_srt #labeling the columns ...
co_matrix

------------------------------------------------------------------------------
*   **Step abandoned [crashed for trying to display all the pairs in matrix]**
------------------------------------------------------------------------------


In [23]:
#display all the pairs in the matrix
#don't run --> it will crash
#don't run --> it will crash
#don't run --> it will crash

for i in co_matrix.index: 
  for j in co_matrix.columns:
    co_matrix[i][j] = (str(i)+str(j))

co_matrix

<ipython-input-23-6c4c8f3a536a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  co_matrix[i][j] = (str(i)+str(j))


KeyboardInterrupt: ignored

### **Constructing the Covisitation Matrix** 
**Filling the Covisitation Matrix** (time: 29s from the last run)

In [ ]:
for key, value in dict(pairs_counter).items():
   a = key.split(",")[0].strip()
   b = key.split(",")[1].split(":")[0].strip()
   c = value

   co_matrix.loc[a,b] = c ###### it may have writen c to cell [a, b], but it is not showing in the table below

co_matrix


In [ ]:
#for checking only (i.e. the top 50x50 matrix)
co_matrix[:50][:50]

In [ ]:
#for checking only (i.e. checking the values of a, b, c) 

for key, value in dict(pairs_counter).items():
   a = key.split(",")[0].strip()
   b = key.split(",")[1].split(":")[0].strip()
   c = value

   print (a, b, c)

### **Predicting**

*   **Normalizing the matrix (could be skipped)** (time: 4s from the last run)




In [ ]:
from sklearn.preprocessing import normalize #TO CONFIRM (How it should be normalized?) ALSO, cosine_similarity with or without the normalizing the matrix seem to give the same result

co_matrix_norm = pd.DataFrame(normalize(co_matrix))
co_matrix_norm.index = unique_val_srt #label the rows with the list of unique item ID
co_matrix_norm.columns = unique_val_srt #label the columns ...

co_matrix_norm

*   **Cosine_similarity_scores (with normalized matrix)** (time: 2m4s from the last run)







In [56]:
from sklearn.metrics.pairwise import cosine_similarity #[cosine_similarity (with normalized matrix)]

cos_score_df = pd.DataFrame(cosine_similarity(co_matrix_norm)) #compute similarity score with cosine_similarity (without normalized the matrix)
cos_score_df.index = unique_val_srt #label the rows with the list of unique item ID
cos_score_df.columns = unique_val_srt #label the columns ...

cos_score_df

,1,2,3,4,5,6,7,8,9,10,...,175475,175579,175589,175773,175775,175777,175779,175945,176165,176271
1,1.000000,0.199041,0.069245,0.033521,0.077152,0.180646,0.080731,0.014290,0.036629,0.175743,...,0.0,0.0,0.006038,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.199041,1.000000,0.134896,0.044544,0.125633,0.164764,0.184516,0.058321,0.089548,0.146228,...,0.0,0.0,0.023002,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.069245,0.134896,1.000000,0.050661,0.528917,0.590083,0.567912,0.029480,0.122374,0.055332,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.033521,0.044544,0.050661,1.000000,0.116470,0.062167,0.078292,0.043644,0.058441,0.077279,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.077152,0.125633,0.528917,0.116470,1.000000,0.522006,0.625761,0.020607,0.055188,0.041743,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175777,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0
175779,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,1.0,0.0
175945,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0
176165,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [59]:
new_user = np.zeros(len(co_matrix), dtype=int) #create a vector as a new user entry
random_index = np.random.randint(0, len(co_matrix)) #make a random index

new_user[random_index]=1 #generate 1 for a random-th row
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1

len(new_user) == len(co_matrix)
new_user

array([0, 0, 0, ..., 0, 0, 0])

*   **Predicting (making recommendations) for a new user**

In [63]:
A = new_user
B = cos_score_df.to_numpy()

out_asc = np.sort(np.matmul(A, B))
out_desc = out_asc[::-1] #sort the score in desc for top similarty scores
recommendation_all = [] 
candidates_only = []

for i in range(len(new_user)):
  if new_user[i] == 0:
    recommendation_all.append(out_desc[i])
    candidates_only.append(out_desc[i])
  if new_user[i] > 0: 
    recommendation_all.append(str(out_desc[i]) + ' implied preference') #mark those that are already preferred by the user

recommendation_all[:100] #view the top 100 candidates (including implied preference)
candidates_only[:100] #view the top 100 candidates

[1.0000000000000002,
 0.447213595499958,
 0.34729169384182046,
 0.316227766016838,
 0.2581988897471612,
 0.223606797749979,
 0.20000000000000004,
 0.1825741858350554,
 0.1825741858350554,
 0.16903085094570336,
 0.16903085094570333,
 0.16514456476895414,
 0.158113883008419,
 0.15060017387484667,
 0.14907119849998599,
 0.14142135623730953,
 0.14142135623730953,
 0.12792042981336627,
 0.1267731382092775,
 0.1240347345892085,
 0.12247448713915891,
 0.12171612389003697,
 0.11952286093343938,
 0.11952286093343938,
 0.11952286093343938,
 0.11547005383792518,
 0.11462410798513055,
 0.1118033988749895,
 0.1084652289093281,
 0.105409255338946,
 0.105409255338946,
 0.10446597681448266,
 0.10259783520851543,
 0.10000000000000002,
 0.1,
 0.09534625892455924,
 0.0932504808240314,
 0.0932504808240314,
 0.0912870929175277,
 0.0912870929175277,
 0.0912870929175277,
 0.08944271909999157,
 0.08770580193070295,
 0.08451542547285168,
 0.08451542547285167,
 0.08451542547285167,
 0.08451542547285167,
 0.0827